In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import torchvision

import os
import sys
sys.path.insert(0, "../utils")
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm

import cityscapes_loader
import train_eval
import importlib

In [2]:
importlib.reload(cityscapes_loader)

dataset_root_dir = "/home/nfs/inf6/data/datasets/cityscapes/"

train_ds = cityscapes_loader.cityscapesLoader(root=dataset_root_dir, split='train', img_size=(512, 1024), is_transform=True, is_sequence=True)
val_ds = cityscapes_loader.cityscapesLoader(root=dataset_root_dir, split='val', img_size=(512, 1024), is_transform=True, is_sequence=True)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=5, shuffle=True, drop_last=True)
valid_loader = torch.utils.data.DataLoader(val_ds, batch_size=5, shuffle=False, drop_last=True)

Found 1885 train images
Found 235 val images


In [3]:
import UNet_Template
import importlib
importlib.reload(UNet_Template)

config = UNet_Template.RNN_UNetConfig(out_channels=train_ds.n_classes+1)

rnn_unet = UNet_Template.RNN_UNet(config)

rnn_unet_optim = torch.optim.Adam(rnn_unet.parameters(), lr=3e-4)

criterion = nn.CrossEntropyLoss()

In [4]:
importlib.reload(train_eval)

epochs=100
rnn_unet_trainer = train_eval.Trainer(
    rnn_unet, rnn_unet_optim, criterion, train_loader, valid_loader, "cityscapes", epochs, sequence=False, all_labels=20, start_epoch=0)

/home/user/schwemme/CudaLab_Project/src/tboard_logs/RNN_UNet


In [5]:
rnn_unet_trainer.train_model()

FileNotFoundError: [Errno 2] No such file or directory: '/home/nfs/inf6/data/datasets/cityscapes/gtFine_sequence/val/frankfurt/frankfurt_000000_000019_gtFine_labelIds.png'

In [ ]:
test_label = label[1]
print(test_label.shape)
test_label = val_ds.decode_segmap(test_label.numpy()[0])
# display the image
plt.imshow(test_label)

In [ ]:
def display_seq(seq):
    for i in range(seq.shape[0]):
        img = seq[i]
        mean = torch.tensor([0.485, 0.456, 0.406]).view( 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img = img * std + mean
        img = img.cpu().permute(1, 2, 0).numpy()
        plt.imshow(img)
        # add number 
        plt.title(i)
        plt.show()

display_seq(img)